# Eat Safe, Love

## Notebook Set Up  
##mongoimport --db uk_food --collection establishments --file "C:\Users\Owner\Documents\Data Analytics\homework\nosql-challenge\Resources\establishments.json" --jsonArray

In [ ]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
uk_food = mongo['uk_food']

In [ ]:
# review the collections in our database
print(uk_food.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = uk_food['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
cnt = establishments.count_documents(query)
print(cnt)

# Display the first document in the results using pprint
result = establishments.find_one(query)
pprint(result)

In [ ]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query)
establishmenths_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(establishmenths_df.shape)

# Display the first 10 rows of the DataFrame
establishmenths_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.  Using regular expression to get London
query = {"RatingValue": {"$gte": 4}, "LocalAuthorityName": {"$regex": "London"}}

# Use count_documents to display the number of documents in the result.  Result 33.
cnt = establishments.count_documents(query)
print(cnt)

# Display the first document in the results using pprint
result = establishments.find_one(query)
pprint(result)

In [ ]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query)
establishmentrv_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame.
print(establishmentrv_df.shape)

# Display the first 10 rows of the DataFrame
establishmentrv_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {"RatingValue": 5,
       "geocode.latitude": {"$gte": (latitude - degree_search), "$lt": (latitude + degree_search)},
       "geocode.longitude": {"$gte": (longitude - degree_search), "$lt": (longitude + degree_search)}
        }
sort = [("scores.Hygiene", 1)]
limit = 5

# Print the results
results = list(establishments.find(query).sort(sort).limit(limit))
pprint(results)

In [ ]:
# Convert result to Pandas DataFrame
top5_rv5_df = pd.DataFrame(results)
top5_rv5_df.head()

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Where Clause
match_query = {'$match': 
               {"scores.Hygiene": 0}
              }

# Group By 
group_query = {'$group': 
               {'_id': {"classification": "$LocalAuthorityName", "culture": "$culture"}, 
                'count': { '$sum': 1 }
               }
              }

# Sort
sort_values = {'$sort': { 'count': -1 }}

# Put the pipeline together
pipeline = [match_query, group_query, sort_values]


# Print the number of documents in the result
results = establishments.aggregate(pipeline)
results = list(results)
print(len(results))

# Print the first 10 results
pprint(results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
hs0_df = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(hs0_df.shape)

# Display the first 10 rows of the DataFrame
hs0_df.head(10)